In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as dsplit
from rgf.sklearn import FastRGFClassifier, RGFClassifier
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from keras import regularizers
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K
from sklearn import metrics
from sklearn.metrics import mean_squared_error
%matplotlib inline
plt.style.use('fivethirtyeight')

Using TensorFlow backend.


In [2]:
samples_file = pd.read_csv(r"C:\Users\maher\Desktop\Data_Bhutan\samples.csv")
X = samples_file.drop(['FID', 'CID', 'Type', 'Type_ID'], axis = 1)
y = samples_file['Type_ID']
print(X.shape)
print(y.shape)

(1221, 10)
(1221,)


In [3]:
xtr, xte, ytr, yte = dsplit(X.values, y.values, test_size = 0.5, random_state = 42,stratify = y)
print('Training Features Shape:', xtr.shape)
print('Training Labels Shape:', ytr.shape)
print('Testing Features Shape:', xte.shape)
print('Testing Labels Shape:', yte.shape)

Training Features Shape: (610, 10)
Training Labels Shape: (610,)
Testing Features Shape: (611, 10)
Testing Labels Shape: (611,)


In [45]:

rgf = RGFClassifier(calc_prob='sigmoid',learning_rate=0.01, loss='LS')
rgf = rgf.fit(xtr, ytr)

In [4]:
whole_file = pd.read_csv(r"C:\Users\maher\Desktop\Data_Bhutan\whole.csv")
objectid = whole_file['OBJECTID']
X_whole = whole_file.drop(['OBJECTID', 'pointid', 'grid_code'], axis = 1)
X_whole.shape

(187965, 10)

In [6]:
def normalize(x_tr, x_te, whole):
    mean = np.mean(x_tr)
    std_dev = np.std(x_tr)
    x_norm_tr = (x_tr - mean) / std_dev 
    x_norm_te = (x_te - mean) / std_dev 
    whole_norm = (whole - mean) / std_dev 
    return x_norm_tr, x_norm_te, whole_norm

_, _, X_whole= normalize(xtr, xte, X_whole)
xtr, xte, _= normalize(xtr, xte, _)

In [7]:
class SparseAutoencoder(object):
    
    def __init__(self, features, encoding_dim, opt, hidden_nodes, loss_fn, np_epochs, bsize, vsize):
        self.features = features
        self.encoding_dim = encoding_dim
        self.opt = opt
        self.hidden_nodes = hidden_nodes
        self.loss_fn = loss_fn
        self.np_epochs = np_epochs
        self.bsize = bsize
        self.vsize = vsize
        self.autoencoder = None
        self.encoder = None
        self.create_autoencoder()
        
    def create_autoencoder(self):
        input_features = Input(shape=(self.features.shape[1],))
        
        encoded1 = Dense(self.hidden_nodes, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_features)
        encoded2 = Dense(self.encoding_dim, activation='relu', activity_regularizer=regularizers.l2(10e-5))(encoded1)
        encoded3 = Dense(self.hidden_nodes, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded2)

        decoded = Dense(self.features.shape[1], activation='sigmoid')(encoded3)
        autoencoder = Model(input_features, decoded)
        autoencoder.compile(optimizer=self.opt, loss=self.loss_fn)
        autoencoder.fit(self.features, self.features,epochs=self.np_epochs,batch_size=self.bsize,
                        shuffle=True,validation_split=self.vsize)
        encoder = Model(input = input_features, output = encoded2)
        self.autoencoder = autoencoder
        self.encoder = encoder
        return
    
    def reconstruct_input(self, features):
        reconstructed_features = self.autoencoder.predict(features)
        return reconstructed_features
    
    def extract_compressed_features(self, features):
        c_features = self.encoder.predict(x= features)
        return c_features
    
    def measure_error(self, actual_features, reconstructed_features):
        reconstruction_error = mean_squared_error(actual_features, reconstructed_features)
        return reconstruction_error

In [8]:
def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
def KLD(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.sum(y_true * K.log(y_true / y_pred), axis=-1)

In [13]:
class RGF(object):
    
    def __init__(self, xtr, xte, ytr, yte, lr):
        self.xtr = xtr
        self.xte = xte
        self.ytr = ytr
        self.yte = yte
        self.lr = lr
        self.tr_rgf = self.fit()
        
    def fit(self):
        rgf = RGFClassifier(calc_prob='sigmoid',learning_rate=self.lr, loss='LS')
        rgf = rgf.fit(self.xtr, self.ytr)
        return rgf
    
    def predict(self, x):
        return self.tr_rgf.predict(x)
    
    def predict_proba(self,x):
        return self.tr_rgf.predict_proba(x)
    
    def measure_accuracy(self, x, true_y):
        preds = self.predict(x)
        fpr, tpr, thresholds = metrics.roc_curve(true_y, preds, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        print('AUC = ', np.round(auc, 3))
        return
        
    def rank_features(self, X):
        feature_list = list(X.columns)
        importances = list(self.tr_rgf.feature_importances_)
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        x_values = list(range(len(importances)))
        plt.bar(x_values, importances, orientation = 'vertical')
        plt.xticks(x_values, feature_list, rotation='vertical')
        plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');   
        return

In [15]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

class TreeModels(object):
    def __init__(self, xtr, xte, ytr, yte, model):
        self.xtr = xtr
        self.xte = xte
        self.ytr = ytr
        self.yte = yte
        self.model = model 
        self.trained_model = self.fit()
        
    def fit(self):
        dt = DecisionTreeClassifier()
        rf = RandomForestClassifier()
        et = ExtraTreeClassifier()
        dt = dt.fit(self.xtr, self.ytr)
        rf = rf.fit(self.xtr, self.ytr)
        et = et.fit(self.xtr, self.ytr)
        if self.model == 'dt':
            return dt
        elif self.model == 'rf':
            return rf
        else:
            return et
    
    def predict(self, x):
        return self.trained_model.predict(x)
        
    def predict_proba(self,x):
        return self.trained_model.predict_proba(x)
    
    def measure_accuracy(self, x, true_y):
        preds = self.predict(x)
        fpr, tpr, thresholds = metrics.roc_curve(true_y, preds, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        print('AUC = ', np.round(auc, 3))
        return
        
    def rank_features(self, X):
        feature_list = list(X.columns)
        importances = list(self.trained_model.feature_importances_)
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        x_values = list(range(len(importances)))
        plt.bar(x_values, importances, orientation = 'vertical')
        plt.xticks(x_values, feature_list, rotation='vertical')
        plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances : ' + self.model);   
        return

In [41]:
sa = SparseAutoencoder(xtr, 10, 'adamax', 14, 'binary_crossentropy', 800, 32, 0.01)

Train on 603 samples, validate on 7 samples
Epoch 1/800
603/603 [==============================] - 1s 938us/step - loss: 0.6006 - val_loss: 0.5093
Epoch 2/800
603/603 [==============================] - 0s 30us/step - loss: 0.3833 - val_loss: 0.2378
Epoch 3/800
603/603 [==============================] - 0s 44us/step - loss: 0.0403 - val_loss: -0.1740
Epoch 4/800
603/603 [==============================] - 0s 52us/step - loss: -0.4620 - val_loss: -0.7403
Epoch 5/800
603/603 [==============================] - 0s 42us/step - loss: -1.1620 - val_loss: -1.4887
Epoch 6/800
603/603 [==============================] - 0s 49us/step - loss: -2.1113 - val_loss: -2.5604
Epoch 7/800
603/603 [==============================] - 0s 44us/step - loss: -3.4811 - val_loss: -4.1113
Epoch 8/800
603/603 [==============================] - 0s 44us/step - loss: -4.6440 - val_loss: -5.1147
Epoch 9/800
603/603 [==============================] - 0s 42us/step - loss: -4.9348 - val_loss: -5.2092
Epoch 10/800
603/603 [==

603/603 [==============================] - 0s 36us/step - loss: -6.7265 - val_loss: -5.7655
Epoch 79/800
603/603 [==============================] - 0s 35us/step - loss: -6.7447 - val_loss: -5.7799
Epoch 80/800
603/603 [==============================] - 0s 33us/step - loss: -6.7627 - val_loss: -5.8113
Epoch 81/800
603/603 [==============================] - 0s 36us/step - loss: -6.7814 - val_loss: -5.8248
Epoch 82/800
603/603 [==============================] - 0s 39us/step - loss: -6.8002 - val_loss: -5.8463
Epoch 83/800
603/603 [==============================] - 0s 45us/step - loss: -6.8181 - val_loss: -5.8619
Epoch 84/800
603/603 [==============================] - 0s 44us/step - loss: -6.8350 - val_loss: -5.8720
Epoch 85/800
603/603 [==============================] - 0s 43us/step - loss: -6.8494 - val_loss: -5.8985
Epoch 86/800
603/603 [==============================] - 0s 39us/step - loss: -6.8643 - val_loss: -5.9089
Epoch 87/800
603/603 [==============================] - 0s 40us/step

603/603 [==============================] - 0s 30us/step - loss: -7.3014 - val_loss: -6.2136
Epoch 156/800
603/603 [==============================] - 0s 32us/step - loss: -7.3054 - val_loss: -6.2113
Epoch 157/800
603/603 [==============================] - 0s 31us/step - loss: -7.3084 - val_loss: -6.2166
Epoch 158/800
603/603 [==============================] - 0s 39us/step - loss: -7.3124 - val_loss: -6.2205
Epoch 159/800
603/603 [==============================] - 0s 38us/step - loss: -7.3161 - val_loss: -6.2194
Epoch 160/800
603/603 [==============================] - 0s 39us/step - loss: -7.3190 - val_loss: -6.2236
Epoch 161/800
603/603 [==============================] - 0s 56us/step - loss: -7.3223 - val_loss: -6.2257
Epoch 162/800
603/603 [==============================] - 0s 30us/step - loss: -7.3262 - val_loss: -6.2262
Epoch 163/800
603/603 [==============================] - 0s 40us/step - loss: -7.3299 - val_loss: -6.2277
Epoch 164/800
603/603 [==============================] - 0s 

603/603 [==============================] - 0s 38us/step - loss: -7.5313 - val_loss: -6.3935
Epoch 233/800
603/603 [==============================] - 0s 31us/step - loss: -7.5345 - val_loss: -6.3996
Epoch 234/800
603/603 [==============================] - 0s 42us/step - loss: -7.5372 - val_loss: -6.3941
Epoch 235/800
603/603 [==============================] - 0s 35us/step - loss: -7.5398 - val_loss: -6.3987
Epoch 236/800
603/603 [==============================] - 0s 29us/step - loss: -7.5428 - val_loss: -6.4012
Epoch 237/800
603/603 [==============================] - 0s 30us/step - loss: -7.5455 - val_loss: -6.4063
Epoch 238/800
603/603 [==============================] - 0s 26us/step - loss: -7.5486 - val_loss: -6.4086
Epoch 239/800
603/603 [==============================] - 0s 30us/step - loss: -7.5510 - val_loss: -6.4083
Epoch 240/800
603/603 [==============================] - 0s 32us/step - loss: -7.5535 - val_loss: -6.4135
Epoch 241/800
603/603 [==============================] - 0s 

603/603 [==============================] - 0s 37us/step - loss: -7.6460 - val_loss: -6.5318
Epoch 310/800
603/603 [==============================] - 0s 32us/step - loss: -7.6467 - val_loss: -6.5355
Epoch 311/800
603/603 [==============================] - 0s 43us/step - loss: -7.6478 - val_loss: -6.5362
Epoch 312/800
603/603 [==============================] - 0s 38us/step - loss: -7.6479 - val_loss: -6.5379
Epoch 313/800
603/603 [==============================] - 0s 44us/step - loss: -7.6493 - val_loss: -6.5370
Epoch 314/800
603/603 [==============================] - 0s 44us/step - loss: -7.6500 - val_loss: -6.5400
Epoch 315/800
603/603 [==============================] - 0s 30us/step - loss: -7.6507 - val_loss: -6.5384
Epoch 316/800
603/603 [==============================] - 0s 37us/step - loss: -7.6518 - val_loss: -6.5398
Epoch 317/800
603/603 [==============================] - 0s 43us/step - loss: -7.6522 - val_loss: -6.5395
Epoch 318/800
603/603 [==============================] - 0s 

...

603/603 [==============================] - 0s 32us/step - loss: -7.7961 - val_loss: -6.7047
Epoch 541/800
603/603 [==============================] - 0s 37us/step - loss: -7.7967 - val_loss: -6.7054
Epoch 542/800
603/603 [==============================] - 0s 34us/step - loss: -7.7969 - val_loss: -6.7034
Epoch 543/800
603/603 [==============================] - 0s 35us/step - loss: -7.7976 - val_loss: -6.7077
Epoch 544/800
603/603 [==============================] - 0s 35us/step - loss: -7.7979 - val_loss: -6.7086
Epoch 545/800
603/603 [==============================] - 0s 32us/step - loss: -7.7983 - val_loss: -6.7075
Epoch 546/800
603/603 [==============================] - 0s 36us/step - loss: -7.7988 - val_loss: -6.7065
Epoch 547/800
603/603 [==============================] - 0s 34us/step - loss: -7.7992 - val_loss: -6.7078
Epoch 548/800
603/603 [==============================] - 0s 38us/step - loss: -7.7996 - val_loss: -6.7062
Epoch 549/800
603/603 [==============================] - 0s 

603/603 [==============================] - 0s 30us/step - loss: -7.8193 - val_loss: -6.7113
Epoch 618/800
603/603 [==============================] - 0s 35us/step - loss: -7.8193 - val_loss: -6.7112
Epoch 619/800
603/603 [==============================] - 0s 29us/step - loss: -7.8197 - val_loss: -6.7128
Epoch 620/800
603/603 [==============================] - 0s 41us/step - loss: -7.8200 - val_loss: -6.7166
Epoch 621/800
603/603 [==============================] - 0s 36us/step - loss: -7.8203 - val_loss: -6.7150
Epoch 622/800
603/603 [==============================] - 0s 38us/step - loss: -7.8206 - val_loss: -6.7105
Epoch 623/800
603/603 [==============================] - 0s 30us/step - loss: -7.8203 - val_loss: -6.7149
Epoch 624/800
603/603 [==============================] - 0s 35us/step - loss: -7.8208 - val_loss: -6.7129
Epoch 625/800
603/603 [==============================] - 0s 33us/step - loss: -7.8210 - val_loss: -6.7109
Epoch 626/800
603/603 [==============================] - 0s 

603/603 [==============================] - 0s 35us/step - loss: -7.8360 - val_loss: -6.7217
Epoch 695/800
603/603 [==============================] - 0s 38us/step - loss: -7.8363 - val_loss: -6.7266
Epoch 696/800
603/603 [==============================] - 0s 32us/step - loss: -7.8365 - val_loss: -6.7191
Epoch 697/800
603/603 [==============================] - 0s 33us/step - loss: -7.8367 - val_loss: -6.7176
Epoch 698/800
603/603 [==============================] - 0s 35us/step - loss: -7.8368 - val_loss: -6.7158
Epoch 699/800
603/603 [==============================] - 0s 32us/step - loss: -7.8367 - val_loss: -6.7230
Epoch 700/800
603/603 [==============================] - 0s 34us/step - loss: -7.8372 - val_loss: -6.7233
Epoch 701/800
603/603 [==============================] - 0s 34us/step - loss: -7.8374 - val_loss: -6.7253
Epoch 702/800
603/603 [==============================] - 0s 39us/step - loss: -7.8373 - val_loss: -6.7214
Epoch 703/800
603/603 [==============================] - 0s 

603/603 [==============================] - 0s 29us/step - loss: -7.8493 - val_loss: -6.7286
Epoch 772/800
603/603 [==============================] - 0s 37us/step - loss: -7.8490 - val_loss: -6.7254
Epoch 773/800
603/603 [==============================] - 0s 39us/step - loss: -7.8491 - val_loss: -6.7296
Epoch 774/800
603/603 [==============================] - 0s 32us/step - loss: -7.8494 - val_loss: -6.7290
Epoch 775/800
603/603 [==============================] - 0s 33us/step - loss: -7.8496 - val_loss: -6.7313
Epoch 776/800
603/603 [==============================] - 0s 32us/step - loss: -7.8497 - val_loss: -6.7299
Epoch 777/800
603/603 [==============================] - 0s 35us/step - loss: -7.8501 - val_loss: -6.7306
Epoch 778/800
603/603 [==============================] - 0s 35us/step - loss: -7.8502 - val_loss: -6.7292
Epoch 779/800
603/603 [==============================] - 0s 32us/step - loss: -7.8499 - val_loss: -6.7279
Epoch 780/800
603/603 [==============================] - 0s 

C:\Users\maher\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [42]:
r_xtr = sa.reconstruct_input(xtr)
r_xte = sa.reconstruct_input(xte)

c_xtr = sa.extract_compressed_features(xtr)
c_xte = sa.extract_compressed_features(xte)
error1 = sa.measure_error(xtr,r_xtr)
error2 = sa.measure_error(xte,r_xte)
print(np.round(error1, 3), np.round(error2, 3))

0.534 0.528


In [43]:
rgf1 = RGF(xtr, xte, ytr, yte, 0.01)
rgf2 = RGF(c_xtr, xte, ytr, yte, 0.01)
print('ARGF')
auc3 = rgf2.measure_accuracy(c_xtr, ytr)
auc4 = rgf2.measure_accuracy(c_xte, yte)

ARGF
AUC =  0.968
AUC =  0.73


In [16]:
rgf1 = RGF(xtr, xte, ytr, yte, 0.01)
rgf2 = RGF(c_xtr, xte, ytr, yte, 0.01)

dt1 = TreeModels(xtr, xte, ytr, yte, 'dt')
dt2 = TreeModels(c_xtr, xte, ytr, yte, 'dt')

rf1 = TreeModels(xtr, xte, ytr, yte, 'rf')
rf2 = TreeModels(c_xtr, xte, ytr, yte, 'rf')

et1 = TreeModels(xtr, xte, ytr, yte, 'et')
et2 = TreeModels(c_xtr, xte, ytr, yte, 'et')

print('RGF')
auc1 = rgf1.measure_accuracy(xtr, ytr)
auc2 = rgf1.measure_accuracy(xte, yte)

print('ARGF')
auc3 = rgf2.measure_accuracy(c_xtr, ytr)
auc4 = rgf2.measure_accuracy(c_xte, yte)

print('DT')
auc5 = dt1.measure_accuracy(xtr, ytr)
auc6 = dt1.measure_accuracy(xte, yte)
auc7 = dt2.measure_accuracy(c_xtr, ytr)
auc8 = dt2.measure_accuracy(c_xte, yte)

print('RF')
auc9 = rf1.measure_accuracy(xtr, ytr)
auc10 = rf1.measure_accuracy(xte, yte)
auc11 = rf2.measure_accuracy(c_xtr, ytr)
auc12 = rf2.measure_accuracy(c_xte, yte)

print('ET')
auc13 = et1.measure_accuracy(xtr, ytr)
auc14 = et1.measure_accuracy(xte, yte)
auc15 = et2.measure_accuracy(c_xtr, ytr)
auc16 = et2.measure_accuracy(c_xte, yte)

C:\Users\maher\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\maher\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\maher\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\maher\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\maher\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

RGF
AUC =  0.963
AUC =  0.895
ARGF
AUC =  0.972
AUC =  0.765
DT
AUC =  1.0
AUC =  0.83
AUC =  1.0
AUC =  0.735
RF
AUC =  0.994
AUC =  0.844
AUC =  0.976
AUC =  0.724
ET
AUC =  1.0
AUC =  0.792
AUC =  1.0
AUC =  0.752


In [ ]:
rgf.rank_features(X)

In [16]:
r_whole = sa.reconstruct_input(X_whole)
c_whole = sa.extract_compressed_features(X_whole)
error1 = sa.measure_error(X_whole,r_whole)
error1

0.6076470092509515

In [17]:
rgfx = rgf1.predict_proba(X_whole)
argfx = rgf2.predict_proba(c_whole)

In [46]:
dtx = dt2.predict_proba(c_whole)
rfx = rf2.predict_proba(c_whole)
etx = et2.predict_proba(c_whole)

In [49]:
etx

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [325]:
d = np.vstack((objectid, rgfx[:,0]))
np.savetxt(r"C:\Users\maher\Desktop\Data_Bhutan\rgf.csv", d.T, delimiter=",")

In [329]:
d2 = np.vstack((objectid, argfx[:,0]))
np.savetxt(r"C:\Users\maher\Desktop\Data_Bhutan\rgf2.csv", d2.T, delimiter=",")

In [22]:
d3 = np.vstack((objectid, dtx[:,0]))
np.savetxt(r"C:\Users\maher\Desktop\Data_Bhutan\dt.csv", d3.T, delimiter=",")

In [50]:
d4 = np.vstack((objectid, rfx[:,0]))
np.savetxt(r"C:\Users\maher\Desktop\Data_Bhutan\rf.csv", d4.T, delimiter=",")

In [24]:
d5 = np.vstack((objectid, etx[:,0]))
np.savetxt(r"C:\Users\maher\Desktop\Data_Bhutan\et.csv", d5.T, delimiter=",")